# CH 12-13

## TOC<a id='toc'></a>
* [Ch12 Notes](#ch12_notes)
* [Ch13 Notes](#ch13_notes)

### CH12 Notes <a id='ch12_notes'></a>
[toc](#toc)
### Inheritance: For Good or For Worse

### Subclassing Built-in is Tricky
* code for built-ins (written in C) does not call sepcial methods overwritten by user
* built-in behavior is a violation of a basic rule of object-oriented programming: 
    - the search for methods should always start from the class of the target instance (self) even when the call happens inside a method implemented in a super class.
* instead of subclassing built-ins, derive your classes from collections module, like `UserDict`, etc, which are meant to be extended.

### Multiple Inheritance and Method Resolution Order
* the **diamong problem**: naming conflicts arising from multiple inheritance, when unrelated ancestors implement method by same name
    - *can call methods direclty from class, by passing instance var as argument*
* amibuguity resolved becuase python follows a specific order when traversing the inheritance graph - called the **MRO**: **Method Reslution Order**
    - classes have an attribute called `__mro__` holding a tuple of references to the superclasses in MRO order, from current, all the way to the `object` class.
    - takes into account not only inheritance, but also order in which superclasses are listed in the subclass declaration
* recomended way to delegate method casll to superclass is using the `super()` built-in
    - will follow MRO
* can bypass by using desired class name directly, and passing self. 
    - (you must pass self in this case because you are accessing an **unbound method**)
    - this is not recommendend

### Multiple Inheritance in the Real World
* Adapter pattern uses it (only one of the 22 patterns in the Design Patterns book)
* Tkinter uses it - but it is 20 yrs old, and no longer best practice
* Django does it well

### Coping with multiple inheritance
1. Distinguish interface inheritance from implementation inheritance
    - two reasons to inherit: 1) create a "is-a" relationship (interface inheritance) and 2) avoid code duplication (implementation inheritance)
    - in practice, often both. But when often you can replace just the second with composition or delegation
2. Make interfaces explicit with ABCs
3. Use Mixins for code reuse
    - conceptually, a mixin does not define a new type; it merely bundles code for reuse.
    - mixin should never be instantiated, and a concrete class should not inherit only from a mixin
    - each mixin should provide a specific behavior, implementing few and very closely related methods.
4. Make mixinx explicit by naming
5. ABC can be a mixin, but not the other way around
    - however, abc concrete methods should use only the public interface; while this need not be true of mixin
6. Don't subclass from more than one concrete class
    - some people say not even one concrete class: "all non-leaf classes should be abstract" -Scott Meyer's
7. Provide Aggregate classes to Users
    - create empty classes that are useful combinations of mixins and/or abcs (called **aggregate classes**)
8. VIP: <font color='blue'> Favor object composition over class inheritance </font>
    - inheritance is a very tight coupling, and even with single inheritance, tall trees tend to be very brittle 
    - composition enhances flexibility

<hr>
<br>
Interesting paradigm: if many things inherit from ABC, that don't all need to implement the same methods, don't include them in abc, but include some "dispatch" method which can check existence and delegate to appropriate "handler" methods, implemented by concrete classes (or raise).
<br>
<hr>

Always ask: "Am I writing an application, or a framework?" - If application, the classes you code are generally leaf classes. If not, you are probably reinventing the wheel, and there is some framework out there that does what you need.

### CH13 Notes <a id='ch13_notes'></a>
[toc](#toc)
### Operator Overloading: Doing it Right

Python operator overloading limitations:
* cannot overload operators for builtins
* cannot create new operators
* some operators cannot be overloaded (`is`, `and`, `or`, `not`)
    - but bitwise ( `&, |, ~`) can

### Unary Operators
* `-`  `__neg__`: arithmetic negation. `-x`
* `+` `__pos__`: arithmetic unary plus
    - ???
* `~` `__invert__`: bitwise inverse of integer: defined as `~x = -(x+1)`
* Some consider `abs(...)` as unary op too - `__abs__`

Overloading
- just implement special method
- fundamental rule of operators: always return a new object, do not modify self.

### Binary Operators
* `+` `__add__(self, other)`

To support operations involving objects of diferent types, Python implements a special dispatching mechanism for the infix operatprs special methods: given `a + b`, interpreter will perform thses steps
- if `a` has `__add__`, call `a.__add__(b)`, and return result unless its `NotImplemented`
    * **Do not confuse `NotImplemented` -  a special singleton value - with `NotImplementedError` - an exception
- if `a` doesn't have it, or returned notimplemented, check if `b` has `__radd__`. If so call, and return unless returns NotImplementd
- if `b` doesnt have it or returned notImplemented, raise `TypeError` with unsuported operand types message

* `__radd__` - r can stand for reversed, or reflected, or right.

In [3]:
raise NotImplemented

TypeError: exceptions must derive from BaseException

In [4]:
raise NotImplementedError

NotImplementedError: 

Good practice to catch error and return `NotImplemented`, so that python can call the other guys radd. Otherwise error stops the dispatching.
    - do this in the `__add__`
    - in the `__radd__`, you should raise reasonable error

Jargon:
- infix means operator is between operands, as opposed to prefix or postfix
    * 2+2, vs +2 2 or 2 2 +

### Infix operators (less rich comparison)
* +, - , *, /, //, divmod(), ** (or pow()), &, |, ^, <<, >>

Python 3.5 now has an extra operator: `@`
    - its associated special methods are: `__matmul__`, `__rmatmul__`, and `__imatmul__` (this last one is @=)

### Rich comparison operators
- ==, !=, >, <, >=, <=
- different than above in two important ways
    * same set called in forward and reverse calls (but `__lt__` switches to `__gt__`)
    * in the case of `==` and `!=`, if reverse call fails, then python comparse ids
- often no need for `__ne__`, because fall back behavior is to negate result of `__eq__`

### Augmented Assignment Operators
* augmented assignment operators must return self (prolly not code enforced)
* if the class doesnt implement the augmented assignment operators, then `a += b` is evaluated as `a = a + b`
    - obviously, in place special methods should never be implemented for immutable types
* note generally `+=` is more liberal in its rhs than just `+`, because no ambiguity as to what return type is desired.

Operator overloading is one area in which isinstance checks are common - but it is preferred to use goose typing